In [1]:
from torch.utils.data import DataLoader
import os
import torch
import sys

from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA

from auxilary.utils import *
from dataset import nucleiDataset, nucleiValDataset, nucleiTestDataset
from networkModules.modelUnet3p import UNet_3Plus
from auxilary.lossFunctions import *


In [2]:
config = readConfig('configs/config_cryonuseg.sys')

In [3]:
#trainPaths = config["trainDataset"]
trainPaths = "/mnt/Datasets/MoNuSeg/Training/sampleImages/"
valPaths = config["valDataset"]
testPaths = config["testDataset"]
train_dataset = nucleiDataset(trainPaths, config)
val_dataset = nucleiValDataset(valPaths, config)
test_dataset = nucleiTestDataset(testPaths, config)
trainLoader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=False)
val_data = DataLoader(val_dataset,batch_size=1,num_workers=4)
test_data = DataLoader(test_dataset,batch_size=1)

In [4]:
model = UNet_3Plus(config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = weightedDiceLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=config["learning_rate"], weight_decay=1e-5)

In [5]:
epochs = 1
for epoch in range(epochs):
    for batch in tqdm(trainLoader):
        
        image, mask, samencoding = batch

        print(f"Image shape: {image.shape}")
        print(f"Mask shape: {mask.shape}")
        print(f"Samencoding shape: {samencoding.shape}")

        
        gt = mask.squeeze().float()

        gt = gt.type(torch.float32)

        outputs = model((image.to(device), samencoding.to(device)))

        sys.exit(0)
        
        loss = criterion(outputs, gt.to(device))

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

        #print(f"Loss: {loss.item()}")


        pass

    print(f"Epoch {epoch+1}/{epochs} completed.")

  0%|          | 0/2 [00:00<?, ?it/s]

Image shape: torch.Size([16, 3, 256, 256])
Mask shape: torch.Size([16, 2, 256, 256])
Samencoding shape: torch.Size([16, 256, 64, 64])


  0%|          | 0/2 [00:00<?, ?it/s]


SystemExit: 0

/home/bishal/projects/SAM-Guided-U-Net/samGEnv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


torch.Size([1, 256, 64, 64])
